# Introducción a la manipulación de datos con pandas

![](https://pandas.pydata.org/_static/pandas_logo.png)

[Pandas](https://pandas.pydata.org/) es una biblioteca que provee herramientas para trabajar con `estructuras de datos`, `análisis de datos`, con alto rendimiento, pero fácil de usar.

* Permite leer y escrbir archivos.
* Organizar, realizar operaciones, seccionar, modificar datos de manera eficiente.
* Mostrarlos de forma amigable.
* Con fines académicos y comerciales, en financias, neurociencia, economía, estadística, publicidad, análisis web, etc.


Este material se inspira en el tutorial de [**Ariel Rossanigo**](https://twitter.com/arielrossanigo) en PyData San Luis. [Github Ariel](https://github.com/arielrossanigo)

## Temario

* Series y Dataframes
* Lectura de datos
* Indexado
* Operaciones
* Merge
* Group
* Plots
* Tips and tricks

### Breve repaso de Numpy

* Arrays multidimensionales implementados de forma eficiente
* Base para muchos de los paquetes científicos en Python


**Ejecutar la siguiente celda**

In [ ]:
! conda install -y matplotlib pandas

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("Funciona!")

### Estructuras básicas

* **Serie**: Array unidimensional indexado y etiquetado.

In [ ]:
nombres = pd.Series(['john', 'paul', 'george', 'ringo']) # array indexado
nombres

* **Dataframe**: Array bidimensional etiquetado con datos estructurados en columnas de potencialmente diferentes tipos.

![](https://storage.googleapis.com/lds-media/images/series-and-dataframe.width-1200.png)
*Extraído de [Python Pandas tutorial](https://www.learndatasci.com/tutorials/python-pandas-tutorial-complete-introduction-for-beginners/)*

*Estructura de datos cuyas columnas se componen por series*

In [ ]:
beatles = pd.DataFrame({    
    'nombre': nombres, # Columna formada por la serie definida antes
    'nacimiento': [1940, 1942, 1943, 1940] # Nueva columna
})
beatles

**EJERCICIO 0**: Crear un Dataframe como el anterior pero con 2 columnas extras:

* instrumento: en orden serían ('guitarra', 'bajo', 'guitarra', 'bateria')
* permanencia: (9, 10, 10, 8)

### Leyendo datos

**Pandas** viene preparado para interactuar con varios formatos de datos, entre ellos **CSV, Excel, HDF5, pickle, SQL** y varios más. Algunos de los parámetros más usados de `read_csv`:

**read_csv(argumentos)**:

* filepath_or_buffer: ruta al archivo (string).
* usecols: columnas a leer.
* parse_dates: columnas a parsear como fechas.
* dtype: tipos de datos de las columnas.
* na_values: valores que son considerados "NA".

[Documentación read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html#pandas.read_csv)

Vamos a trabajar con datos expuestos por Organismos del Gobierno de la República Argentina, más precisamente por el Ministerio de Ciencia y Tecnología.

Vamos a usar 2 datasets:

* Proyectos de ciencia, tecnología e innovación
* Empresas de ejecución de proyectos de ciencia, tecnología e innovación

In [ ]:
proyectos = pd.read_csv('proyectos.csv')
proyectos.head(10)

In [ ]:
# Un poco de detalle
proyectos.info()

**EJERCICIO 1**: Leer nuevamente el csv, pero:

 * recuperar solamente las columnas: 'proyecto_id', 'fecha_inicio', 'provincia_de_ejecución', 'monto_financiado', 'monto_total', 'gran_area_conocimiento', 'tipo_organizacion_ejec'
 * la columna ``fecha_inicio`` debe ser parseada como date (formato de fecha).

### Filtrado de datos

* Por etiqueta: **loc**
* Por posición: **iloc**
* Indexado condicional: *a la numpy*

In [ ]:
proyectos.loc[1:3, 'fecha_inicio':'monto_total'] 
# Filas 1 a 3 inclusive, desde columnas fecha inicio hasta monto total

In [ ]:
proyectos.iloc[1:3, 0:3]
# Filas 1 a 3 inclusive, columnas 0 a 2 

In [ ]:
proyectos = pd.read_csv('proyectos.csv', parse_dates=['fecha_inicio'])

In [ ]:
proyectos[(proyectos['provincia_de_ejecución'] == 'San Luis') &
          (proyectos.fecha_inicio.dt.year == 2016)]
# Datos del dataframe proyectos donde se cumpla que
# 'provincia_de_ejecucion' es 'San Luis'
# Y
# año de fecha de inicio es 2016

**EJERCICIO 2**: Mostrar los proyectos de CABA, que se hayan financiado con más de 1MM de pesos.

### Agregado de columnas, operaciones básicas

In [ ]:
# la forma más simple, con operaciones entre series
proyectos['porcentaje_financiado'] = proyectos.monto_financiado / proyectos.monto_total
proyectos.head(10)

* **np.where**: `np.where`(condición, valor_para_true, valor_para_false)

In [ ]:
# valor condicional
proyectos['financia_mas_80_por_ciento'] = np.where(proyectos.porcentaje_financiado > 0.8, 'Si', 'No')
proyectos.head(15)

### Estadística

* `mean()`: calcula el valor medio.
* `std()`: calcula la desviación.

In [ ]:
print("Monto de proyecto. Promedio: {:,.2f} $. Desvio: {:,.2f} $".format(
    proyectos.monto_total.mean(),
    proyectos.monto_total.std()
))

* `describe()`: Proporciona información estadística completa para todo el dataframe

In [ ]:
s = pd.Series([1, 2, 3])
s.describe()

In [ ]:
proyectos.describe(include='all')

* **value_counts()**: retorna cuántos valores cumplen algún criterio.

In [ ]:
# ¿Cuántos proyectos por provincia?
proyectos.provincia_de_ejecución.value_counts().head(5)

* **pd.cut**: clasifica datos según criterio.

In [ ]:
# bins de montos de proyecto (rangos fijos)
bins = [0, 1e6, 2e7, 1e20]
# 0 - 1_000_000 barato
# 1_000_000 - 20_000_000 normal
# 20_000_000 - 100_000_000_000_000_000_000 caro
names = ['Barato', 'Normal', 'Caro']

proyectos['costo'] = pd.cut(proyectos.monto_total, bins, labels=names)
proyectos.costo.value_counts()

In [ ]:
bins = [0, .33, .66, 1]
proyectos['costo'] = pd.qcut(proyectos.monto_total, bins, labels=names)
proyectos.costo.value_counts()

### Aplicando funciones

* `lambda x: funcion de x`: retorna en procesamiento de x.

In [ ]:
proyectos.monto_total.apply(lambda x: '{:,.2f} $'.format(x)).head(3)

In [ ]:
proyectos.apply(lambda x: x.monto_total - x.monto_financiado, axis='columns').head(3)

### Funciones con strings

Hay un atributo *str* para tal fin

In [ ]:
# pasar a minúsculas
proyectos.tipo_organizacion_ejec.str.lower().head(3)

# contiene universidad o ciencia
ix = proyectos.tipo_organizacion_ejec.str.contains('universidad|ciencia', case=False)
proyectos[ix].tipo_organizacion_ejec.unique()

### Algunos métodos útiles

* **drop_duplicates**: si hay varias filas repetidas deja sólo una (no tiene en cuenta el índice)
* **fillna**: completa con el valor que recibe como parámetro las celdas sin valor.


In [ ]:
proyectos.gran_area_conocimiento.fillna('???').unique()

In [ ]:
proyectos.gran_area_conocimiento.drop_duplicates()

**EJERCICIO 3**:

* ¿Cuál es el área de conocimiento con más proyectos?
* ¿Qué porcentaje del costo se financia en promedio?

### Agrupando datos

Involucra uno o más de los siguientes pasos:

* **Separar** los datos en grupos en base a algún criterio
* **Aplicar** una función a cada grupo de forma independiente

 * Aggregation
 * Transformation
 * Filtration

* **Combinar** los resultados en una estructura de datos

https://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby

In [ ]:
# cantidad de proyectos y promedio de monto financiado por provincia
(proyectos.groupby(proyectos['provincia_de_ejecución'])
 .monto_financiado.agg(['mean', 'count'])
 .head(10))

In [ ]:
# los 3 proyectos con mayor financiacion por provincia
ordenado = proyectos.sort_values(by=['provincia_de_ejecución', 'monto_financiado'], 
                                 ascending=False)
ordenado.groupby('provincia_de_ejecución').head(3).head(10)

**EJERCICIO 4**: En el dataset se puede apreciar que hay más de un registro por proyecto. Esto se debe a que el mismo proyecto puede estar en más de una provincia a la vez o abarcar más de un área de conocimiento.

* ¿Cuánto es el monto total financiado en cada año sabiendo lo antes mencionado? 

### Combinando datos

#### Concat

    pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
              keys=None, levels=None, names=None, verify_integrity=False,
              copy=True)


In [ ]:
pd.concat([beatles, beatles], axis=1)

### Combinando datos

#### Merge

    pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)


In [ ]:
muertes = pd.DataFrame({
    'nombre': ['john', 'george'],
    'año de muerte': [1980, 2001] 
})

pd.merge(beatles, muertes, on='nombre')

In [ ]:
m2 = muertes.set_index('nombre')

pd.merge(beatles, m2, 
         left_on='nombre', right_index=True, how='left', 
         indicator=True, validate='one_to_one')

In [ ]:
instrumentos = pd.DataFrame({
    'nombre': ['john', 'john', 'ringo',  'ringo', 'charly'],
    'instrumento': ['guitarra', 'teclado', 'bateria', 'percusión', 'piano'] 
})

pd.merge(beatles, instrumentos, 
         left_on='nombre', right_on='nombre', how='outer', 
         indicator=True, validate='one_to_many')

**EJERCICIO 5**: Usar el dataset de empresas provisto debajo para determinar el top 5 de empresas en cuanto a su involucración en los  proyectos de mayor monto

In [ ]:
empresas = pd.read_csv('empresas.csv', parse_dates=['fecha_inicio'])
empresas.head(3)

### Ploteando datos

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2015', periods=1000))
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=['A', 'B', 'C', 'D'])
df = df.cumsum()
df.head(10)

In [ ]:
df.plot(figsize=(12, 4));

In [ ]:
f, axis = plt.subplots(1, 2, figsize=(12, 4))
df.boxplot(ax=axis[0])
df.A.hist(ax=axis[1]);

**EJERCICIO 6**:

* ¿Cuánto es el monto total financiado año a año? 
* ¿Cuánto es el monto total financiado provincia? 

In [ ]:
años = empresas.fecha_inicio.dt.year

### Algunos consejos 

* Evitar usar utilizar ``apply`` => Tratar de usar operaciones sobre vectores
* Evitar ``iterrows`` => Acceder 
* ``concat`` duplica el consumo de memoria al momento de la concatenación => Depende del caso, HDF5 quizás ayuda